In [1]:
import sys
import os
sys.path.insert(0, os.path.realpath(".."))
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import copy
import numpy as np

DATASET = 'D'
WORK_DIR = Path(f"../data/{DATASET}")
drift_ranges = [1709449200, 1709488800, 1709571600, 1711548600, 1711574700, 1711599900, 1711611600, 1711634400, 1711682400, 1712052000, 1712080800, 1712116800, 1712145600, 1712167200, 1712188800, 1712491200, 1712530800, 1712595600, 1712682000, 1712854800, 1712867400]
fault_df = pd.read_csv(WORK_DIR / "faults.csv")
with open(WORK_DIR / "drift.txt", 'r') as f:
    drift_ts = int(f.read())
print(f"{drift_ts=}")
metric_df = pd.read_pickle(WORK_DIR / "metrics.norm.pkl")
failure_timestamps = []
for _, failure in fault_df.iterrows():
    timestamp = failure['timestamp']
    failure_timestamps.extend([timestamp + i*60 for i in range(0, 20)])

metric_index_dict = metric_df.groupby(['name']).groups
metric_index_after_dict = metric_df[metric_df.timestamp>=drift_ts & ~(metric_df.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metric_index_before_dict = metric_df[(metric_df.timestamp<drift_ts) & ~(metric_df.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metrics_before = set(metric_index_before_dict.keys())
metrics_after = set(metric_index_after_dict.keys())
metrics = list(metrics_before & metrics_after)
metrics = [metric for metric in metrics if 'loadgenerator' not in metric]
print(f"metrics_before: {len(metrics_before)}, metrics_after: {len(metrics_after)}, metrics: {len(metrics)}")

drift_ts=1709442000
metrics_before: 4437, metrics_after: 4737, metrics: 4258


In [2]:
import json
with open(f'../data/{DATASET}/system_change.json', 'r') as f:
    drifts = json.load(f)
drifts = [drift[0] for drift in drifts][1:]
print(drifts)
failure_ts = []
for _, failure in fault_df.iterrows():
    failure_ts.append(failure['timestamp'])
print(failure_ts)

['traffic_0.2', 'traffic_2', 'order_svc_delay', 'traffic_0.2', 'station_svc_cpu', 'travel_svc_memory', 'order_svc_delay', 'node3_node_cpu', 'node3_node_memory', 'order_svc_cpu', 'preserve_svc_memory', 'seat_svc_delay', 'node3_node_cpu', 'node4_node_memory', 'traffic_0.6', 'order_svc_memory', 'traffic_2', 'node2_node_cpu', 'travel_svc_cpu', 'node2_node_memory']
[1709139600, 1709143200, 1709146800, 1709161200, 1709164800, 1709168400, 1709218800, 1709220600, 1709226000, 1709227800, 1709229600, 1709231400, 1709233200, 1709235000, 1709236800, 1709238600, 1709240400, 1709242200, 1709244000, 1709245800, 1709247600, 1709249400, 1709253000, 1709254800, 1709256600, 1709265600, 1709267400, 1709269200, 1709271000, 1709272800, 1709276400, 1709278200, 1709280000, 1709281800, 1709283600, 1709285400, 1709287200, 1709289000, 1709290800, 1709292600, 1709294400, 1709296200, 1709298000, 1709299800, 1709301600, 1709303400, 1709305200, 1709307000, 1709316000, 1709317800, 1709319600, 1709326800, 1709328600, 

In [3]:
from bayesian_changepoint_detection.priors import const_prior
from functools import partial
from bayesian_changepoint_detection.bayesian_models import offline_changepoint_detection
import bayesian_changepoint_detection.offline_likelihoods as offline_ll
from pathlib import Path

pdf_root = Path('../figures/change_metric_figures')
pdf_root.mkdir(exist_ok=True)

CP_THRESHOLD = 0.7

metric_df_icpp = copy.deepcopy(metric_df)
metric_index_dict = metric_df_icpp.groupby(['name']).groups
# metric_index_before_dict = metric_df_icpp[(metric_df_icpp.timestamp<drift_ts) & ~(metric_df_icpp.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metric_index_before_dict = metric_df_icpp[(metric_df_icpp.timestamp<drift_ts)].groupby(['name']).groups
metrics_before = set(metric_index_before_dict.keys())

Use scipy logsumexp().


## Bayesian Change Point Detection & adaption

In [4]:
from bayesian_changepoint_detection.priors import const_prior
from functools import partial
from bayesian_changepoint_detection.bayesian_models import offline_changepoint_detection
import bayesian_changepoint_detection.offline_likelihoods as offline_ll
import time

CP_THRESHOLD = 0.7

metric_df_icpp = copy.deepcopy(metric_df)
metric_index_dict = metric_df_icpp.groupby(['name']).groups
metric_index_before_dict = metric_df_icpp[(metric_df_icpp.timestamp<drift_ts) & ~(metric_df_icpp.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metrics_before = set(metric_index_before_dict.keys())

time_detect, time_adapt = [], []

for i in range(len(drift_ranges)-1):
    drift_num = 0
    drift_metrics = []
    drift_start, drift_end = drift_ranges[i], drift_ranges[i+1]
    print(f"{drift_start=} {drift_end=}")
    metric_index_after_dict = metric_df_icpp[(metric_df_icpp.timestamp>=drift_start) & (metric_df_icpp.timestamp<drift_end) & ~(metric_df_icpp.timestamp.isin(failure_timestamps))].groupby(['name']).groups
    metric_index_after_all_dict = metric_df_icpp[(metric_df_icpp.timestamp>=drift_start) & (metric_df_icpp.timestamp<drift_end)].groupby(['name']).groups
    metrics_after = set(metric_index_after_dict.keys())
    metrics = list(metrics_before & metrics_after)
    metrics = [metric for metric in metrics if 'loadgenerator' not in metric]
    print(f"metrics_before: {len(metrics_before)}, metrics_after: {len(metrics_after)}, metrics: {len(metrics)}")
    
    is_drifts, change_points, length_list = [], [], []
    for metric in tqdm(metrics, desc="drift metrics: "):
        if 'fake' in metric:
            continue
        metric_index = metric_index_dict[metric]
        metric_index_after = metric_index_after_dict[metric]
        metric_index_before = metric_index_before_dict[metric]
        metrics_data = metric_df_icpp.loc[metric_index]

        before_metrics = metrics_data.loc[metric_index_before].sort_values(by=['timestamp']).drop_duplicates()[-120:]
        metrics_data_after = metrics_data.loc[metric_index_after]
        after_metrics = metrics_data_after.sort_values(by=['timestamp']).drop_duplicates()[:90]
        if before_metrics.empty or after_metrics.empty or len(after_metrics) < 10:
            continue
        before_metrics_value = before_metrics.value
        after_metrics_value = after_metrics.value

        combined_array = np.concatenate((before_metrics_value[-20:],after_metrics_value[:20]))
        time_start = time.time()
        Q, P, Pcp = offline_changepoint_detection(combined_array, partial(const_prior, p=1/(len(combined_array) + 1)) ,offline_ll.StudentT(),truncate=-40)
        a = np.exp(Pcp).sum(0)
        threshold_indices = np.where(a > CP_THRESHOLD)[0] + 2  # 满足阈值的位置，并加2
        filtered_change_points = []
        last_accepted_cp = -np.inf
        for cp in threshold_indices:
            if cp >= len(before_metrics_value[-20:]) and (last_accepted_cp == -np.inf or cp - last_accepted_cp >= 3):
                if last_accepted_cp != -np.inf and cp - last_accepted_cp < 3:
                    continue
                filtered_change_points.append(cp)
                last_accepted_cp = cp 
        if filtered_change_points:
            change_point = filtered_change_points[-1]  # 取最后一个位置
            is_drift = True
        else:
            change_point = None  # 无有效变点
            is_drift = False
        time_end = time.time()
        time_detect.append(time_end-time_start)
        is_drifts.append(is_drift)
        change_points.append(change_point)
        length_list.append((len(before_metrics_value), len(after_metrics_value)))
        if is_drift:
            time_start = time.time()
            before_mean = before_metrics_value.mean()
            before_std = before_metrics_value.std()
            if change_point is not None:
                offset_in_after = change_point - len(before_metrics_value[-20:])
                print(f"change_point: {change_point}, offset_in_after: {offset_in_after}")
                after_metrics_value = after_metrics_value[offset_in_after+1:]
            after_mean = after_metrics_value.mean()
            after_std = after_metrics_value.std()
            drift_num += 1
            drift_metrics.append(metric)
            scale = before_std / after_std if after_std != 0 else 1
            bias = before_mean - after_mean * scale

            metric_index_after_all = metric_index_after_all_dict[metric]
            metrics_data_after_all = metrics_data.loc[metric_index_after_all]
            after_metrics_value_all = metrics_data_after_all['value'].values * scale + bias
            metric_df_icpp.loc[metric_index_after_all, 'value'] = after_metrics_value_all
            time_end = time.time()
            time_adapt.append(time_end-time_start)
    print(f"successfully drift {drift_num} metrics using method [icpp] in range ({drift_start}, {drift_end})")
    print(f"\n\n{sum(is_drifts)} drifts of all {len(metrics)} metrics...")

metric_df_icpp.to_pickle(WORK_DIR / f'metrics.norm.detect.drift.mean.pkl')

drift_start=1709449200 drift_end=1709488800
metrics_before: 4437, metrics_after: 4263, metrics: 4234


drift metrics:   0%|          | 13/4234 [00:15<14:43,  4.78it/s]  

change_point: 21, offset_in_after: 1


drift metrics:   0%|          | 15/4234 [00:15<12:07,  5.80it/s]

change_point: 40, offset_in_after: 20


...

change_point: 35, offset_in_after: 15


drift metrics:  99%|█████████▉| 4206/4234 [11:07<00:04,  6.52it/s]

change_point: 38, offset_in_after: 18


drift metrics: 100%|██████████| 4234/4234 [11:12<00:00,  6.30it/s]

successfully drift 257 metrics using method [icpp] in range (1709449200, 1709488800)


257 drifts of all 4234 metrics...


## w/o CPD

In [ ]:
metric_df_wo_cpd = copy.deepcopy(metric_df)
metric_index_dict = metric_df_wo_cpd.groupby(['name']).groups
metric_index_before_dict = metric_df_wo_cpd[(metric_df_wo_cpd.timestamp<drift_ts) & ~(metric_df_wo_cpd.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metrics_before = set(metric_index_before_dict.keys())
for i in range(len(drift_ranges)-1):
    drift_num = 0
    drift_metrics = []
    drift_start, drift_end = drift_ranges[i], drift_ranges[i+1]
    print(f"{drift_start=} {drift_end=}")
    metric_index_after_dict = metric_df_wo_cpd[(metric_df_wo_cpd.timestamp>=drift_start) & (metric_df_wo_cpd.timestamp<drift_end) & ~(metric_df_wo_cpd.timestamp.isin(failure_timestamps))].groupby(['name']).groups
    metric_index_after_all_dict = metric_df_wo_cpd[(metric_df_wo_cpd.timestamp>=drift_start) & (metric_df_wo_cpd.timestamp<drift_end)].groupby(['name']).groups
    metrics_after = set(metric_index_after_dict.keys())
    metrics = list(metrics_before & metrics_after)
    metrics = [metric for metric in metrics if 'loadgenerator' not in metric]
    print(f"metrics_before: {len(metrics_before)}, metrics_after: {len(metrics_after)}, metrics: {len(metrics)}")
    
    for metric in tqdm(metrics, desc="drift metrics: "):
        if 'fake' in metric:
            continue
        metric_index = metric_index_dict[metric]
        metric_index_after = metric_index_after_dict[metric]
        metric_index_before = metric_index_before_dict[metric]
        metrics_data = metric_df_wo_cpd.loc[metric_index]
        before_metrics = metrics_data.loc[metric_index_before].sort_values(by=['timestamp']).drop_duplicates()[-120:]
        metrics_data_after = metrics_data.loc[metric_index_after]
        after_metrics = metrics_data_after.sort_values(by=['timestamp']).drop_duplicates()[:90]
        if before_metrics.empty or after_metrics.empty or len(after_metrics) < 10 or len(after_metrics.value.unique()) == 1:
            continue
        before_metrics_value = before_metrics.value
        before_median = before_metrics_value.median()
        before_IQR = before_metrics_value.quantile(0.75) - before_metrics_value.quantile(0.25)
        after_metrics_value = after_metrics.value
        after_median = after_metrics_value.median()
        after_IQR = after_metrics_value.quantile(0.75) - after_metrics_value.quantile(0.25)
        drift_num += 1
        drift_metrics.append(metric)
        scale = before_IQR / after_IQR if after_IQR != 0 else 1
        bias = before_median - after_median * scale

        metric_index_after_all = metric_index_after_all_dict[metric]
        metrics_data_after_all = metrics_data.loc[metric_index_after_all]
        after_metrics_value_all = metrics_data_after_all['value'].values * scale + bias
        metric_df_wo_cpd.loc[metric_index_after_all, 'value'] = after_metrics_value_all
    print(f"successfully drift {drift_num} metrics using method [wo_cpd] in range ({drift_start}, {drift_end})")
metric_df_wo_cpd.to_pickle(WORK_DIR / f'metrics.norm.detect.mean.pkl')

## w StepWise

In [ ]:
from sklearn.linear_model import RANSACRegressor

metric_df_stepwise = copy.deepcopy(metric_df)
metric_index_dict = metric_df_stepwise.groupby(['name']).groups
metric_index_before_dict = metric_df_stepwise[(metric_df_stepwise.timestamp<drift_ts) & ~(metric_df_stepwise.timestamp.isin(failure_timestamps))].groupby(['name']).groups
metrics_before = set(metric_index_before_dict.keys())
for i in range(len(drift_ranges)-1):
    drift_num = 0
    drift_metrics = []
    drift_start, drift_end = drift_ranges[i], drift_ranges[i+1]
    print(f"{drift_start=} {drift_end=}")
    metric_index_after_all_dict = metric_df_stepwise[(metric_df_stepwise.timestamp>=drift_start) & (metric_df_stepwise.timestamp<drift_end)].groupby(['name']).groups
    metric_index_after_dict = metric_df_stepwise[(metric_df_stepwise.timestamp>=drift_start) & (metric_df_stepwise.timestamp<drift_end) & ~(metric_df_stepwise.timestamp.isin(failure_timestamps))].groupby(['name']).groups
    metrics_after = set(metric_index_after_dict.keys())
    metrics = list(metrics_before & metrics_after)
    metrics = [metric for metric in metrics if 'loadgenerator' not in metric]
    print(f"metrics_before: {len(metrics_before)}, metrics_after: {len(metrics_after)}, metrics: {len(metrics)}")
    
    for metric in tqdm(metrics, desc="drift metrics: "):
        if 'fake' in metric:
            continue
        metric_index = metric_index_dict[metric]
        metric_index_after = metric_index_after_dict[metric]
        metric_index_before = metric_index_before_dict[metric]
        metrics_data = metric_df_stepwise.loc[metric_index]
        before_metrics = metrics_data.loc[metric_index_before].sort_values(by=['timestamp']).drop_duplicates()
        metrics_data_after = metrics_data.loc[metric_index_after]
        after_metrics = metrics_data_after.sort_values(by=['timestamp']).drop_duplicates()[:30]
        if before_metrics.empty or after_metrics.empty or len(after_metrics) < 10 or len(after_metrics.value.unique()) == 1:
            continue
        before_metrics_value = before_metrics.value
        
        ts_medians = np.zeros(24*60//17, dtype=np.float32)
        mod = 5040      # 24*60*60//17
        for i in range(24*60//17):
            tmp = before_metrics.loc[before_metrics.timestamp % mod == i*60].sort_values(by=['timestamp']).drop_duplicates()[-10:]['value']
            ts_medians[i] = tmp.median(skipna=True)
        ts_medians[np.isnan(ts_medians)] = 0
        target_series = []
        for i in range(after_metrics.shape[0]):
            tmp = after_metrics.iloc[i]
            timestamp = tmp['timestamp']
            target_series.append(ts_medians[(timestamp%mod)//60])
        target_series = np.array(target_series)
        rlr = RANSACRegressor()
        rlr.fit(np.array(after_metrics.value).reshape((-1, 1)), target_series)

        metric_index_after_all = metric_index_after_all_dict[metric]
        metrics_data_after_all = metrics_data.loc[metric_index_after_all]
        after_metrics_value_all = rlr.predict(metrics_data_after_all['value'].values.reshape((-1, 1)))
        metric_df_stepwise.loc[metric_index_after_all, 'value'] = after_metrics_value_all
    print(f"successfully drift {drift_num} metrics using method [stepwise] in range ({drift_start}, {drift_end})")
metric_df_stepwise.to_pickle(WORK_DIR / f'metrics.norm.drift.stepwise.pkl')